<a href="https://colab.research.google.com/github/sayanbanerjee32/coursera-gen-ai-course-labs/blob/main/coursera_gen_ai_course_lab_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 \
        evaluate==0.4.0 \
        rouge_score==0.1.2 \
        loralib==0..1.1 \
        peft==0.3.0 --quiet

%pip install -Uq pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 43.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 re

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
hf_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(hf_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [4]:
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npct of trainable model parameters: {(trainable_model_params * 100)/all_model_params}%"
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
pct of trainable model parameters: 100.0%


In [6]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs["input_ids"].to(device),
                    max_new_tokens = 200)[0],
    skip_special_tokens = True
)
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f"Input prompt:\n{dialogue}")
print(dash_line)
print(f"Baseline human summary:\n{summary}")
print(dash_line)
print(f"Model generation - zero shot:\n{output}\n")

---------------------------------------------------------------------------------------------------
Input prompt:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
---------------------------------------------------------------------------------------------------
Baseline human sum

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarise the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt, padding = "max_length", truncation = True,
                                     return_tensors = 'pt').input_ids
    example['labels'] = tokenizer(example['summary'], padding = "max_length", truncation = True,
                                     return_tensors = 'pt').input_ids
    return example
tokenized_datasets = dataset.map(tokenize_function, batched = True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index %100 == 0, with_indices = True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
print(f"Shapes of datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")
print(tokenized_datasets)

Shapes of datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


In [10]:
# helper function cleaning GPU memory
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()
report_gpu()

GPU:0
process       6740 uses      694.000 MB GPU memory


In [11]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs = 1,
    weight_decay = 0.01,
    logging_steps =1,
    max_steps = 1
)
trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,49.500000


TrainOutput(global_step=1, training_loss=49.5, metrics={'train_runtime': 2.8101, 'train_samples_per_second': 2.847, 'train_steps_per_second': 0.356, 'total_flos': 5478058819584.0, 'train_loss': 49.5, 'epoch': 0.06})

In [13]:
instruct_model_path = "./instruct_model_path"
trainer.model.save_pretrained(instruct_model_path)
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_path, torch_dtype = torch.bfloat16).to(device)

In [14]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
original_model_outputs = original_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)

instruct_model_outputs = instruct_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)

print(dash_line)
print(f"Baseline human summary:\n{summary}")
print(dash_line)
print(f"Original model:\n{original_model_text_outputs}")
print(dash_line)
print(f"Instruct model:\n{instruct_model_text_outputs}")


---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Original model:
##Person1: Have you considered upgrading your system? ##Person2: Yes, but you might want to add a painting program to your software. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and
------------------------------------------------------------------------------------

In [15]:
rouge = evaluate.load('rouge')

In [16]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summeries = []
instruct_model_summeries = []

for dialogue in dialogues:
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    original_model_outputs = original_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200)).to('cpu')
    original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)
    original_model_summeries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200)).to('cpu')
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)
    instruct_model_summeries.append(instruct_model_text_output)

zipped_sumeries = list(zip(human_baseline_summaries,original_model_summeries,instruct_model_summeries))
df=pd.DataFrame(zipped_sumeries, columns = ['human_baseline_summaries','original_model_summeries','instruct_model_summeries'])
df

,human_baseline_summaries,original_model_summeries,instruct_model_summeries
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to go out as a memo to all employe...,#Person1#: I need to take a dictation for you.
1,In order to prevent employees from wasting tim...,Employees who use Instant Messages will receiv...,#Person1#: I need to take a dictation for you.
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is written to employees regarding the...,#Person1#: I need to take a dictation for you.
3,#Person2# arrives late because of traffic jam....,#Person1: I'm finally here. #Person2: I got st...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,The car is a big part of the problem.,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,#Person2: I'm here!,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,Brianna is a good friend of hers.,"#Person1#: Happy birthday, Brian. #Person2#: I..."


In [18]:
original_model_results = rouge.compute(
    predictions = original_model_summeries,
    references = human_baseline_summaries[0:len(original_model_summeries)],
    use_aggregator = True,
    use_stemmer= True
)

instruct_model_results = rouge.compute(
    predictions = instruct_model_summeries,
    references = human_baseline_summaries[0:len(instruct_model_summeries)],
    use_aggregator = True,
    use_stemmer= True
)

print('Original Model:')
print(original_model_results)
print('Instruct Model:')
print(instruct_model_results)

Original Model:
{'rouge1': 0.26883165445665447, 'rouge2': 0.1134935243799199, 'rougeL': 0.2148711334961335, 'rougeLsum': 0.21585954262369358}
Instruct Model:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}


In [20]:
print("Absolute percentage improvement of Instruct Model over HUman baseline")
improvemnet = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvemnet):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of Instruct Model over HUman baseline
rouge1: -2.79%
rouge2: 0.42%
rougeL: 0.51%
rougeLsum: 0.55%


In [21]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r = 32,
    lora_alpha=32,
    target_modules=["q","w"],
    lora_dropout=0.05,
    bias="none",
    task_type = TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [22]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1769472
all model parameters: 249347328
pct of trainable model parameters: 0.7096414524241463%


In [23]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 1,
    logging_steps =1,
    max_steps = 1
)
peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset=tokenized_datasets['train'],
)

In [24]:
peft_trainer.train()

peft_model_path = "./peft_model_path"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,48.250000


('./peft_model_path/tokenizer_config.json',
 './peft_model_path/special_tokens_map.json',
 './peft_model_path/tokenizer.json')

In [25]:
from peft import PeftModel, PeftConfig

peft_model_base =  AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype = torch.bfloat16,
                                       is_trainable = False).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [26]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 249347328
pct of trainable model parameters: 0.0%


In [28]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
original_model_outputs = original_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
original_model_text_outputs = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)

instruct_model_outputs = instruct_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
instruct_model_text_outputs = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)

peft_model_outputs = peft_model.generate(input_ids = input_ids,generation_config = GenerationConfig(max_new_tokens = 200, num_beams = 1))
peft_model_text_outputs = tokenizer.decode(peft_model_outputs[0],skip_special_tokens = True)

print(dash_line)
print(f"Baseline human summary:\n{summary}")
print(dash_line)
print(f"Original model:\n{original_model_text_outputs}")
print(dash_line)
print(f"Instruct model:\n{instruct_model_text_outputs}")
print(dash_line)
print(f"PEFT model:\n{peft_model_text_outputs}")

---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Original model:
#Person1#: Have you considered upgrading your computer?
---------------------------------------------------------------------------------------------------
Instruct model:
#Person1#: I'm thinking of upgrading my computer.
---------------------------------------------------------------------------------------------------
PEFT model:
#Person1#: I'm thinking of upgrading my computer.


In [29]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summeries = []
instruct_model_summeries = []
peft_model_summeries = []

for dialogue in dialogues:
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    original_model_outputs = original_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200)).to('cpu')
    original_model_text_output = tokenizer.decode(original_model_outputs[0],skip_special_tokens = True)
    original_model_summeries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids,generation_config = GenerationConfig(max_new_tokens = 200)).to('cpu')
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0],skip_special_tokens = True)
    instruct_model_summeries.append(instruct_model_text_output)

    peft_model_outputs = peft_model.generate(input_ids = input_ids,generation_config = GenerationConfig(max_new_tokens = 200)).to('cpu')
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0],skip_special_tokens = True)
    peft_model_summeries.append(peft_model_text_output)

zipped_sumeries = list(zip(human_baseline_summaries,original_model_summeries,instruct_model_summeries, peft_model_summeries))
df=pd.DataFrame(zipped_sumeries, columns = ['human_baseline_summaries','original_model_summeries','instruct_model_summeries', 'peft_model_summeries'])
df

,human_baseline_summaries,original_model_summeries,instruct_model_summeries,peft_model_summeries
0,Ms. Dawson helps #Person1# to write a memo to ...,This memo is to all employees.,#Person1#: I need to take a dictation for you.,The memo is to be distributed to all employees...
1,In order to prevent employees from wasting tim...,"Ms. Dawson, please send a dictation to all emp...",#Person1#: I need to take a dictation for you.,The memo is to be distributed to all employees...
2,Ms. Dawson takes a dictation for #Person1# abo...,Memo is required for all intra-office communic...,#Person1#: I need to take a dictation for you.,The memo is to be distributed to all employees...
3,#Person2# arrives late because of traffic jam....,#Person1#: I'm sorry to hear that.,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1#: I'm sorry to hear that.,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,"#Person1#: I'm here. I'm sorry, I'm not sure w...",The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,#Person1: It's a good thing that they are gett...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are having a separation for 2 m...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is today.,"#Person1#: Happy birthday, Brian. #Person2#: I...",Brian's birthday is coming up.


In [30]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions = original_model_summeries,
    references = human_baseline_summaries[0:len(original_model_summeries)],
    use_aggregator = True,
    use_stemmer= True
)

instruct_model_results = rouge.compute(
    predictions = instruct_model_summeries,
    references = human_baseline_summaries[0:len(instruct_model_summeries)],
    use_aggregator = True,
    use_stemmer= True
)

peft_model_results = rouge.compute(
    predictions = peft_model_summeries,
    references = human_baseline_summaries[0:len(instruct_model_summeries)],
    use_aggregator = True,
    use_stemmer= True
)

print('Original Model:')
print(original_model_results)
print('Instruct Model:')
print(instruct_model_results)
print('PEFT Model:')
print(peft_model_results)

Original Model:
{'rouge1': 0.22130752743009113, 'rouge2': 0.054214729370008874, 'rougeL': 0.2021182782734507, 'rougeLsum': 0.20448599713629695}
Instruct Model:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}
PEFT Model:
{'rouge1': 0.26109650997150996, 'rouge2': 0.11055072463768116, 'rougeL': 0.2302777777777778, 'rougeLsum': 0.2339245014245014}
